In [3]:
from gensim.models import word2vec
import numpy as np

G:\ganda\conda\tensorflow_test\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [41]:
#word2vecモデルの作成と保存
def make_model(text_data, model_name):
    data = word2vec.LineSentence(text_data)
    model = word2vec.Word2Vec(data, size = 200, window=10, hs= 1, sg=1)
    model.save(model_name)
    return model

#学習したモデルから各文章のベクトル化
def document_vector(text, model, num_features):
    model = word2vec.Word2Vec.load(model)
    bag_of_centroids = np.zeros(num_features, dtype = 'float32')
    
    for word in text:
        try:
            temp = model[word]
            #print(temp)
        except:
            continue
        bag_of_centroids += temp
        #print("//")
        #print(bag_of_centroids)
        
    bag_of_centroids = bag_of_centroids / len(text)
    return bag_of_centroids

In [15]:
N_model = make_model("Datas/N_extract_tweet.txt", "Datas/noun_tweet.model")
A_model = make_model("Datas/A_extract_tweet.txt", "Datas/other_tweet.model")

In [30]:
#テスト
model = word2vec.Word2Vec.load('Datas/noun_tweet.model')
index = model.wv.index2word
print(model['陸前高田'])

[-1.87100783e-01  1.09048277e-01  7.84002021e-02  1.41644981e-02
 -1.08856834e-01  2.94806272e-01 -8.46128315e-02 -5.22514954e-02
  2.00934317e-02  3.43402117e-01  9.29339975e-02 -2.20360179e-02
 -3.04976791e-01  6.26765639e-02 -7.81898275e-02 -2.83944444e-03
 -9.34375897e-02  2.52086282e-01 -2.51185987e-02  2.59859681e-01
 -6.28707111e-01  1.61117762e-01 -1.81327872e-02  4.52496260e-01
  6.93801418e-02 -7.46410945e-03 -5.74886650e-02  6.34358302e-02
 -2.61320174e-01 -9.80146378e-02  9.69706327e-02  2.69164424e-02
 -2.99226940e-01 -5.73695540e-01 -2.56332487e-01 -5.06070256e-02
  6.37710094e-02  2.41256416e-01  1.65865451e-01 -2.54726648e-01
  1.94784366e-02  1.44140288e-01  1.38993124e-02  1.88037008e-02
 -1.28342062e-02  9.73312259e-02 -3.29553574e-01 -6.49977088e-01
  1.27824366e-01 -1.19076878e-01 -2.55710900e-01  2.35366434e-01
 -3.20413470e-01  2.12211814e-02 -2.10493952e-01  3.33720684e-01
 -1.24029987e-01 -7.51875490e-02  2.07686909e-02 -1.87826250e-02
 -3.26186329e-01  6.69607

G:\ganda\conda\tensorflow_test\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


In [17]:
def read_data(file):
    f = open(file, "r", encoding="utf-8")
    datas = f.readlines()
    f.close()
    return datas

In [42]:
N_model = "Datas/noun_tweet.model"
wordlen = 200
N_texts = read_data("Datas/N_extract_tweet.txt")
WN_data = [document_vector(a, N_model, wordlen) for a in N_texts]
np_WN_data = np.array(WN_data)

G:\ganda\conda\tensorflow_test\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


In [70]:
from keras.utils.np_utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
#ラベル読み込み
test = open("Datas/label.txt", "r", encoding="utf-8")
label = test.readlines()
test.close()
print(len(label))

training_samples = 8000 # training data 80 : validation data 20
validation_samples = 10000 - training_samples

# バイナリの行列に変換
categorical_labels = to_categorical(label)
labels = np.asarray(categorical_labels)

x_train = np_WN_data[:training_samples]
y_train = labels[:training_samples]
x_val = np_WN_data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

10000


In [66]:
np_WN_data = np.array(WN_data)

In [69]:
print(type(np_WN_data))

<class 'numpy.ndarray'>


In [82]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding
from keras.layers import LSTM
from keras.callbacks import EarlyStopping, TensorBoard
from keras.layers.core import Dropout

model = Sequential()
model.add(Embedding(20000, 100, input_length=200))
model.add(LSTM(1000))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()

early_stopping = EarlyStopping(patience=0, verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 200, 100)          2000000   
_________________________________________________________________
lstm_6 (LSTM)                (None, 1000)              4404000   
_________________________________________________________________
dropout_5 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 2002      
Total params: 6,406,002
Trainable params: 6,406,002
Non-trainable params: 0
_________________________________________________________________


In [83]:
history = model.fit(x_train, y_train,
                    epochs=15, 
                    batch_size=250, 
                    validation_split=0.2, 
                    validation_data=(x_val, y_val))

Train on 8000 samples, validate on 2000 samples
Epoch 1/15
8000/8000 [==============================] - 26s 3ms/step - loss: 0.6936 - acc: 0.5090 - val_loss: 0.6950 - val_acc: 0.4075
Epoch 2/15
8000/8000 [==============================] - 24s 3ms/step - loss: 0.6929 - acc: 0.5194 - val_loss: 0.7122 - val_acc: 0.4075
Epoch 3/15
3000/8000 [==========>...................] - ETA: 13s - loss: 0.6912 - acc: 0.5340

KeyboardInterrupt: 

In [72]:
print(x_train[0])

[-0.05404585  0.01747436  0.06302081  0.10264156  0.04017179  0.05486504
 -0.00281867  0.05627099 -0.05127353  0.04847184  0.05031431 -0.05284579
  0.04907434  0.00873903 -0.01022228 -0.08101268 -0.02625519 -0.03206147
  0.0480625   0.00435327 -0.07044593 -0.03366547 -0.03722711  0.02239307
  0.04952715 -0.00513365  0.00380397  0.00398632  0.01435855  0.00906893
  0.04190715  0.05365361  0.00567958 -0.08920804  0.01804802  0.02728518
  0.02042475 -0.00434983 -0.00316797 -0.04867408 -0.01468666 -0.03402233
  0.01841716 -0.02751425  0.00973431  0.01218706 -0.05285603 -0.0686192
 -0.03201847 -0.03844313  0.00879757  0.07529281 -0.00764451 -0.00065014
 -0.0040061  -0.00138216 -0.0097895  -0.02825337  0.05416796 -0.00215303
 -0.04519954  0.01187047 -0.08524915 -0.00160591  0.03581157  0.0890841
  0.03294931  0.07116871  0.01088624 -0.02185794  0.03700366 -0.01958093
  0.02647966  0.01919383 -0.00857811  0.06232745 -0.03380255 -0.01375844
 -0.04797906 -0.05060933  0.0331441   0.00172181  0.0